In [ ]:
!pip install torch_geometric

In [1]:
import os
# os.environ[“CUDA_DEVICE_ORDER”]=“PCI_BUS_ID”
os.environ["CUDA_VISIBLE_DEVICES"]="1"
# experiment_name = "3_heads_lr3_keep_op_identity+agenda_enriched_all+lr_e3+mult_scalar_per_action+glove"
# experiment_name = "crappy-red-dhole"

from allennlp.models.archival import Archive
import torch
from allennlp.common import Params
from allennlp.models.model import Model
from allennlp.common.params import with_fallback
from dataset_readers.spider_ratsql import SpiderRatsqlDatasetReader
from allennlp.data.vocabulary import Vocabulary


# from models.semantic_parsing.spider_parser import SpiderParser
reader = SpiderRatsqlDatasetReader(tables_file="dataset/tables.json",max_instances=10)
# settings = Params.from_file(f"experiments/{experiment_name}/config.json")
# model = Model.load(config=settings, serialization_dir=f"experiments/{experiment_name}")


train_dataset = reader.read("dataset/train_spider.json")
vocab = Vocabulary.from_instances(train_dataset)

Skipping writing to data cache since max_instances was specified.


In [2]:
# train_dataset = reader.read("dataset/train_spider.json")
from models.semantic_parsing.ratsql_encoder import RatsqlEncoder
# from models.semantic_parsing.gnn_encoder import GnnEncoder
from models.semantic_parsing.spider_decoder import SpiderParser
from allennlp.modules.seq2vec_encoders.boe_encoder import BagOfEmbeddingsEncoder

from allennlp.modules.attention import DotProductAttention
from allennlp.nn.beam_search import BeamSearch
from allennlp.modules.seq2seq_encoders.pass_through_encoder import PassThroughEncoder


from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
# from allennlp.data.iterators import BasicIterator
import torch.optim as optim
from allennlp.training.trainer import Trainer
import torch
# with torch.cuda.device(0):
EMBEDDING_DIM = 200
HIDDEN_DIM = 200
token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})

beam = BeamSearch(end_index=0,beam_size=10)

schema_encoder = RatsqlEncoder(encoder=PassThroughEncoder(200),entity_encoder=BagOfEmbeddingsEncoder(200),question_embedder=word_embeddings,action_embedding_dim=200)
# schema_encoder = GnnEncoder(encoder=PassThroughEncoder(200),entity_encoder=BagOfEmbeddingsEncoder(200),question_embedder=word_embeddings,action_embedding_dim=200)
model = SpiderParser(vocab=vocab,schema_encoder=schema_encoder, 
                     decoder_beam_search=beam,input_attention=DotProductAttention(),past_attention=DotProductAttention(),max_decoding_steps=10)

def get_offsets(lengths):
    e = np.cumsum(([0]+list(lengths))[:-1])
    return list(zip(e+1,e+np.array(lengths)))

#     # model.cuda()
# iterator = BasicIterator(batch_size=15)
# optimizer = optim.Adam(model.parameters(), lr=0.001)
# iterator.index_with(vocab)
# trainer = Trainer(model=model,
#                   optimizer=optimizer,
#                   iterator=iterator,
#                   train_dataset=train_dataset,
#                   validation_dataset=train_dataset,
#                   patience=10,
#                   num_epochs=1)
# trainer.train()

In [3]:
a=None
for inst in train_dataset:
    a = inst
    res_list = model.forward_on_instances([inst])
    break
    
    

ValueError: too many values to unpack (expected 5)

In [4]:
# list()
import numpy as np
b = a.fields['enc']
token_ids = [0]+b.tokens
c = a.fields['lengths'].array

In [16]:
# [0]+token_ids

In [16]:
y = b.tokens[0]
# vars(y)
y.text_id

2129

In [25]:
# print(c.array)
offsets = get_offsets(c)
res = [token_ids[offsets[j][0]:offsets[j][1] + 1] for j in range(len(offsets))]
# print(res)
# print()
# for x in [[vocab.get_token_from_index(y,"tags") for y in x] for x in res]:
for x in res:
    print([y.text_id for y in x])
    print(x)

[2129]
[how]
[2116]
[many]
[4641]
[heads]
[1997]
[of]
[1996]
[the]
[7640]
[departments]
[2024]
[are]
[3080]
[older]
[2084]
[than]
[5179]
[56]
[1029]
[?]
[1026, 2828, 1024, 3793, 1028, 1008, 1026, 2795, 1011, 19802, 1028, 1026, 2151, 1011, 2795, 1028]
[<, type, :, text, >, *, <, table, -, sep, >, <, any, -, table, >]
[1026, 2828, 1024, 2193, 1028, 2533, 8909, 1026, 2795, 1011, 19802, 1028, 2533]
[<, type, :, number, >, department, id, <, table, -, sep, >, department]
[1026, 2828, 1024, 3793, 1028, 2171, 1026, 2795, 1011, 19802, 1028, 2533]
[<, type, :, text, >, name, <, table, -, sep, >, department]
[1026, 2828, 1024, 3793, 1028, 4325, 1026, 2795, 1011, 19802, 1028, 2533]
[<, type, :, text, >, creation, <, table, -, sep, >, department]
[1026, 2828, 1024, 2193, 1028, 5464, 1026, 2795, 1011, 19802, 1028, 2533]
[<, type, :, number, >, ranking, <, table, -, sep, >, department]
[1026, 2828, 1024, 2193, 1028, 5166, 1999, 25501, 1026, 2795, 1011, 19802, 1028, 2533]
[<, type, :, number, >, budg

In [22]:
torch.Size([1, 205, 2])
torch.Size([1, 205, 768])
tensor([[ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, 16, 13, 12, 12, 12, 14, 14,
         13, 12, 13, 12, 13, 13, 13,  4,  4,  4]], dtype=torch.int32) torch.Size([1, 28])
tensor([2129])
tensor([2116])
tensor([4641])
tensor([1997])
tensor([1996])
tensor([7640])
tensor([2024])
tensor([3080])
tensor([2084])
tensor([5179])
tensor([1029])
tensor([ 1026,  2828,  1024,  3793,  1028,  1008,  1026,  2795,  1011, 19802,
         1028,  1026,  2151,  1011,  2795,  1028])
tensor([ 1026,  2828,  1024,  2193,  1028,  2533,  8909,  1026,  2795,  1011,
        19802,  1028,  2533])
tensor([ 1026,  2828,  1024,  3793,  1028,  2171,  1026,  2795,  1011, 19802,
         1028,  2533])
tensor([ 1026,  2828,  1024,  3793,  1028,  4325,  1026,  2795,  1011, 19802,
         1028,  2533])
tensor([ 1026,  2828,  1024,  2193,  1028,  5464,  1026,  2795,  1011, 19802,
         1028,  2533])
tensor([ 1026,  2828,  1024,  2193,  1028,  5166,  1999, 25501,  1026,  2795,
         1011, 19802,  1028,  2533])
tensor([ 1026,  2828,  1024,  2193,  1028, 16371,  1001,  1001,  1049,  5126,
         1026,  2795,  1011, 19802])
tensor([ 1028,  2533,  1026,  2828,  1024,  2193,  1028,  2132,  8909,  1026,
         2795,  1011, 19802])
tensor([ 1028,  2132,  1026,  2828,  1024,  3793,  1028,  2171,  1026,  2795,
         1011, 19802])
tensor([ 1028,  2132,  1026,  2828,  1024,  3793,  1028,  2141,  2110,  1026,
         2795,  1011, 19802])
tensor([ 1028,  2132,  1026,  2828,  1024,  2193,  1028,  2287,  1026,  2795,
         1011, 19802])
tensor([ 1028,  2132,  1026,  2828,  1024,  2193,  1028,  2533,  8909,  1026,
         2795,  1011, 19802])
tensor([ 1028,  2968,  1026,  2828,  1024,  2193,  1028,  2132,  8909,  1026,
         2795,  1011, 19802])
tensor([ 1028,  2968,  1026,  2828,  1024,  3793,  1028,  5741,  3772,  1026,
         2795,  1011, 19802])
tensor([1028, 2968, 1026, 2795])
tensor([1028, 2533, 1026, 2795])
tensor([1028, 2132, 1026, 2795])

SyntaxError: invalid syntax (<ipython-input-22-bcc9a13cbb44>, line 4)

In [2]:
a=None
for inst in train_dataset:
    a = inst
    break

In [4]:
vocab = Vocabulary.from_instances(train_dataset)
b=a.fields['enc']
b.index(vocab)
tok_dict = b._indexed_tokens['tokens']
print(len(tok_dict['token_ids']))


209


In [5]:
import numpy as np
c=a.fields['lengths'].array
token_ids = tok_dict['token_ids']
old_offsets = tok_dict['offsets']
print()

In [90]:
to_str = lambda x: [vocab.get_token_from_index(y,"tags") for y in x]
lengths = np.array(list(c))+1
# offsets = np.cumsum(lengths-1)
# offsets
# print(lengths)

# offsets = list(zip(offsets,offsets)
# offsets

In [88]:
# a_ = 
# b_=offsets+lengths

In [27]:

def get_offsets(lengths):
    e = np.cumsum(([0]+list(lengths))[:-1])
    return list(zip(e+1,e+np.array(lengths)))

offsets = get_offsets(c)

In [28]:
offsets

[(1, 1),
 (2, 2),
 (3, 3),
 (4, 4),
 (5, 5),
 (6, 6),
 (7, 7),
 (8, 8),
 (9, 9),
 (10, 10),
 (11, 11),
 (12, 27),
 (28, 40),
 (41, 52),
 (53, 64),
 (65, 76),
 (77, 90),
 (91, 104),
 (105, 117),
 (118, 129),
 (130, 142),
 (143, 154),
 (155, 167),
 (168, 180),
 (181, 193),
 (194, 197),
 (198, 201),
 (202, 205)]

In [29]:
# _token_indexers
# to_str = lambda x: vocab.get_token_from_index(x,"tags")
# print(c.array)
res = [token_ids[offsets[j][0]:offsets[j][1] + 1] for j in range(len(offsets))]
# print(res)
# print()
for x in [[vocab.get_token_from_index(y,"tags") for y in x] for x in res]:
    print(x)

['how']
['many']
['heads']
['of']
['the']
['departments']
['are']
['older']
['than']
['56']
['?']
['<', 'type', ':', 'text', '>', '*', '<', 'table', '-', 'sep', '>', '<', 'any', '-', 'table', '>']
['<', 'type', ':', 'number', '>', 'department', 'id', '<', 'table', '-', 'sep', '>', 'department']
['<', 'type', ':', 'text', '>', 'name', '<', 'table', '-', 'sep', '>', 'department']
['<', 'type', ':', 'text', '>', 'creation', '<', 'table', '-', 'sep', '>', 'department']
['<', 'type', ':', 'number', '>', 'ranking', '<', 'table', '-', 'sep', '>', 'department']
['<', 'type', ':', 'number', '>', 'budget', 'in', 'billions', '<', 'table', '-', 'sep', '>', 'department']
['<', 'type', ':', 'number', '>', 'nu', '#', '#', 'm', 'employees', '<', 'table', '-', 'sep']
['>', 'department', '<', 'type', ':', 'number', '>', 'head', 'id', '<', 'table', '-', 'sep']
['>', 'head', '<', 'type', ':', 'text', '>', 'name', '<', 'table', '-', 'sep']
['>', 'head', '<', 'type', ':', 'text', '>', 'born', 'state', '<', 

In [74]:
vocab

Vocabulary with namespaces:  rule_labels, Size: 112 || tokens, Size: 2 || Non Padded Namespaces: {'*labels', '*tags'}

In [86]:
vocab.get_token_from_index(102,'tags')
# vocab

'[SEP]'

In [84]:
d= b._token_indexers['tokens']
d

In [83]:
d.indices_to_tokens([23],vocab)

NotImplementedError: 

In [32]:
len()

210

In [12]:
model.forward_on_instances([a])

torch.Size([1, 205, 2])
torch.Size([1, 205, 768])
tensor([[ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, 16, 13, 12, 12, 12, 14, 14,
         13, 12, 13, 12, 13, 13, 13,  4,  4,  4]], dtype=torch.int32) torch.Size([1, 28])


ValueError: too many values to unpack (expected 5)

In [5]:
import random

i=random.randint(0,len(train_dataset))
a=None
for inst in train_dataset[i:i+1]:
    a = inst
    res_list = model.forward_on_instances([inst])
    

In [5]:
a.fields['desc'].metadata

{'raw_question': 'What are the names of the states where at least 3 heads were born?',
 'question': ['What',
  'are',
  'the',
  'names',
  'of',
  'the',
  'states',
  'where',
  'at',
  'least',
  '3',
  'heads',
  'were',
  'born',
  '?'],
 'question_for_copying': ['What',
  'are',
  'the',
  'names',
  'of',
  'the',
  'states',
  'where',
  'at',
  'least',
  '3',
  'heads',
  'were',
  'born',
  '?'],
 'db_id': 'department_management',
 'sc_link': {'q_col_match': {'13,9': 'CPM'}, 'q_tab_match': {}},
 'cv_link': {'num_date_match': {'10,1': 'NUMBER',
   '10,4': 'NUMBER',
   '10,5': 'NUMBER',
   '10,6': 'NUMBER',
   '10,7': 'NUMBER',
   '10,10': 'NUMBER',
   '10,11': 'NUMBER',
   '10,12': 'NUMBER'},
  'cell_match': {}},
 'columns': [['<type: text>', '*', '<table-sep>', '<any-table>'],
  ['<type: number>', 'department', 'id', '<table-sep>', 'department'],
  ['<type: text>', 'name', '<table-sep>', 'department'],
  ['<type: text>', 'creation', '<table-sep>', 'department'],
  ['<type: n

In [6]:
b= res_list[0]['initial_state']
    

In [5]:
c= b.get_valid_actions()

In [ ]:
# c.get_valid_actions()
b.possible_actions

In [ ]:
len(train_dataset)

In [5]:
import random
i=random.randint(0,len(train_dataset))
a=None
for inst in train_dataset[i:i+1]:
#     print(inst)
    a=inst
# b=a.fields['relation']
# b.array
b=a.fields['world']
c=a.fields['desc']
d=a.fields['item'] 
e = a.fields["schema"]
# b
# print(a.fields['utterance'])
for entity in b.metadata.db_context.knowledge_graph.entities:
    print(entity)

column:foreign:management:department_id
column:foreign:management:head_id
column:number:department:budget_in_billions
column:number:department:num_employees
column:number:department:ranking
column:number:head:age
column:primary:department:department_id
column:primary:head:head_id
column:text:department:creation
column:text:department:name
column:text:head:born_state
column:text:head:name
column:text:management:temporary_acting
string:california
string:state
table:department
table:head
table:management


In [6]:
for entity in (c.metadata['columns']+c.metadata['tables']):
def normalize_schema_constant(entity):
#     print(entity)
    col = "_".join(entity)
    col =  col.split("_<table-sep>_")
    if len(col)==1:
        return "_".join(entity)
#         print()
    else:
        table = col[1]
        col = col[0].split(">_")[1]
        return f"{table}@{col}"
#         print()
        
    

['<type: text>', '*', '<table-sep>', '<any-table>']
<any-table>@*
['<type: number>', 'department', 'id', '<table-sep>', 'department']
department@department_id
['<type: text>', 'name', '<table-sep>', 'department']
department@name
['<type: text>', 'creation', '<table-sep>', 'department']
department@creation
['<type: number>', 'ranking', '<table-sep>', 'department']
department@ranking
['<type: number>', 'budget', 'in', 'billions', '<table-sep>', 'department']
department@budget_in_billions
['<type: number>', 'num', 'employees', '<table-sep>', 'department']
department@num_employees
['<type: number>', 'head', 'id', '<table-sep>', 'head']
head@head_id
['<type: text>', 'name', '<table-sep>', 'head']
head@name
['<type: text>', 'born', 'state', '<table-sep>', 'head']
head@born_state
['<type: number>', 'age', '<table-sep>', 'head']
head@age
['<type: number>', 'department', 'id', '<table-sep>', 'management']
management@department_id
['<type: number>', 'head', 'id', '<table-sep>', 'management']
man

In [18]:
e.entity_texts

[[department, id],
 [head, id],
 [budget, in, billions],
 [num, employees],
 [ranking],
 [age],
 [department, id],
 [head, id],
 [creation],
 [name],
 [born, state],
 [name],
 [temporary, acting],
 [rank],
 [department],
 [head],
 [management]]

In [4]:
print(a.fields['valid_actions'])

ListField of 144 ProductionRuleFields : 
 	 ProductionRuleField with rule: arg_list -> [expr, ",", arg_list] (is_global_rule: True) in namespace: 'rule_labels'.' 
 	 ProductionRuleField with rule: arg_list -> [expr] (is_global_rule: True) in namespace: 'rule_labels'.' 
 	 ProductionRuleField with rule: arg_list_or_star -> ["*"] (is_global_rule: True) in namespace: 'rule_labels'.' 
 	 ProductionRuleField with rule: arg_list_or_star -> [arg_list] (is_global_rule: True) in namespace: 'rule_labels'.' 
 	 ProductionRuleField with rule: binaryop -> ["!="] (is_global_rule: True) in namespace: 'rule_labels'.' 
 	 ProductionRuleField with rule: binaryop -> ["*"] (is_global_rule: True) in namespace: 'rule_labels'.' 
 	 ProductionRuleField with rule: binaryop -> ["+"] (is_global_rule: True) in namespace: 'rule_labels'.' 
 	 ProductionRuleField with rule: binaryop -> ["-"] (is_global_rule: True) in namespace: 'rule_labels'.' 
 	 ProductionRuleField with rule: binaryop -> ["/"] (is_global_rule: Tru

In [13]:
d.metadata.schema.columns[3]

Column(id=3, table=Table(id=0, name=['city'], unsplit_name='city', orig_name='city', columns=[Column(id=1, table=..., name=['city', 'id'], unsplit_name='city id', orig_name='City_ID', type='number', foreign_key_for=None), Column(id=2, table=..., name=['official', 'name'], unsplit_name='official name', orig_name='Official_Name', type='text', foreign_key_for=None), ..., Column(id=4, table=..., name=['area', 'km', '2'], unsplit_name='area km 2', orig_name='Area_km_2', type='number', foreign_key_for=None), Column(id=5, table=..., name=['population'], unsplit_name='population', orig_name='Population', type='number', foreign_key_for=None), Column(id=6, table=..., name=['census', 'ranking'], unsplit_name='census ranking', orig_name='Census_Ranking', type='text', foreign_key_for=None)], primary_keys=[Column(id=1, table=..., name=['city', 'id'], unsplit_name='city id', orig_name='City_ID', type='number', foreign_key_for=None)]), name=['status'], unsplit_name='status', orig_name='Status', type='

In [12]:
"asdas_adsdas".split()

['asdas_adsdas']

In [10]:
# print()


[['city'], ['farm'], ['farm', 'competition'], ['competition', 'record']]

In [29]:

# print(b)


In [2]:
import json
from dataset_readers.ratsql_spider_bert import *

# builder = RelationBuilder()


db_path = "dataset/database"
pre = SpiderEncoderV2Preproc()
schemas, eval_foreign_key_maps = pre.load_tables(["dataset/tables.json"])
# for path in paths:
for db_id, schema in tqdm(schemas.items(), desc="DB connections"):
    sqlite_path = Path(db_path) / db_id / f"{db_id}.sqlite"
    source: sqlite3.Connection
    with sqlite3.connect(sqlite_path) as source:
        dest = sqlite3.connect(':memory:')
        dest.row_factory = sqlite3.Row
        source.backup(dest)
    schema.connection = dest
    
examples=[]
raw_data = json.load(open("dataset/train_spider.json"))
for entry in raw_data:
    item = SpiderItem(
        text=entry['question_toks'],
        code=entry['sql'],
        schema=schemas[entry['db_id']],
        orig=entry,
        orig_schema=schemas[entry['db_id']].orig)
#     desc = pre.preprocess_item(item,"train")
    examples.append(item)

        # Backup in-memory copies of all the DBs and create the live connections


DB connections: 100%|██████████| 166/166 [00:00<00:00, 354.40it/s]


In [7]:
# examples
# 
import pandas as pd
print 
# 
import tqdm
for item in tqdm.tqdm(examples):
    desc = pre.preprocess_item(item,"train")
    q = desc['question']
    q_len = len(q)
    t = [x[0] for x in  desc['tables']]
    t_len = len(t)
    c = ["_".join(x) for x in  desc['columns']]
    c_len = len(c)
    enc = q+c+t
    print(enc)
    relation = pre.compute_relations(desc,len(enc),q_len,c_len,range(c_len+1),range(t_len+1))
#     with np.printoptions(threshold=np.inf):
#     print(np.array2string())

    arr = pd.DataFrame(relation)
#     print()
    break
# arr


  0%|          | 0/7000 [00:00<?, ?it/s]

['How', 'many', 'heads', 'of', 'the', 'departments', 'are', 'older', 'than', '56', '?', '<type: text>_*_<table-sep>_<any-table>', '<type: number>_department_id_<table-sep>_department', '<type: text>_name_<table-sep>_department', '<type: text>_creation_<table-sep>_department', '<type: number>_ranking_<table-sep>_department', '<type: number>_budget_in_billions_<table-sep>_department', '<type: number>_num_employees_<table-sep>_department', '<type: number>_head_id_<table-sep>_head', '<type: text>_name_<table-sep>_head', '<type: text>_born_state_<table-sep>_head', '<type: number>_age_<table-sep>_head', '<type: number>_department_id_<table-sep>_management', '<type: number>_head_id_<table-sep>_management', '<type: text>_temporary_acting_<table-sep>_management', 'department', 'head', 'management']


In [8]:
for v in a.items():
    print(v)

('utterance', <allennlp.data.fields.text_field.TextField object at 0x7f36567d44c0>)
('valid_actions', <allennlp.data.fields.list_field.ListField object at 0x7f3660a35fd0>)
('action_sequence', <allennlp.data.fields.list_field.ListField object at 0x7f36565e9730>)
('world', <allennlp.data.fields.metadata_field.MetadataField object at 0x7f36565e9d00>)
('schema', <dataset_readers.fields.spider_knowledge_graph_field.SpiderKnowledgeGraphField object at 0x7f36565e0be0>)


In [10]:
b=a.fields['utterance']

In [14]:
b.sequence_length()
b.tokens

In [20]:
c=a.fields['schema']
c.__dict__

[nltk_data] Downloading package stopwords to /home/ohadr/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
builder.compute_relations(dict(),)

In [10]:
arr.to_csv("t.csv")